In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [1]:
with open("wizard_of_oz_book.txt", "r", encoding="utf-8") as f:
    text = f.read() #text becomes a string variable
    print(len(text))
chars = sorted(set(text))
print(chars)
print(len(chars))
vocabulary_size = len(chars)

233428
['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
83


this shows an array of all the characters used in the book along with the total chars=83, length of all the strings in total are 233428

### encoding and decoding all the chars

In [2]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [3]:
print(encode('hello'))

[63, 60, 67, 67, 70]


we successfully managed to encode characters to digits

In [4]:
encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


This is an example of character-level tokenisation where every char gets tokenised to a digit

In [5]:
import torch

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)#setting torch.long as datatype means we will have a long set of integers
print(data[:100])#printing the first 100 set of integers

tensor([82, 46, 63, 60,  1, 42, 73, 70, 65, 60, 58, 75,  1, 33, 76, 75, 60, 69,
        57, 60, 73, 62,  1, 60, 28, 70, 70, 66,  1, 70, 61,  1, 30, 70, 73, 70,
        75, 63, 80,  1, 56, 69, 59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,
         1, 64, 69,  1, 41, 81,  0,  1,  1,  1,  1,  0, 46, 63, 64, 74,  1, 60,
        57, 70, 70, 66,  1, 64, 74,  1, 61, 70, 73,  1, 75, 63, 60,  1, 76, 74,
        60,  1, 70, 61,  1, 56, 69, 80, 70, 69])


In [7]:
print(len(data))

233428


A tensor is a mathematical object that generalizes scalars, vectors, and matrices to higher dimensions. It is essentially a multi-dimensional array that can represent data in various forms, such as:

    Scalar (0D tensor): A single number (e.g., 5).
    Vector (1D tensor): A one-dimensional array (e.g., [3,7,2][3,7,2]).
    Matrix (2D tensor): A two-dimensional array (e.g., [1234][13​24​]).
    Higher-order tensors (3D and above): Multi-dimensional arrays that extend beyond matrices.

### Splitting into training and validation sets


#### Bigram model

example: hello<br>
start-> h<br>
h-> e<br>
e-> l<br>
l-> l<br>
l-> o <br>
predicting the chance of a character coming prior to which a specific char came. example: prob of e after h

In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
# splitted training testing data in a 80:20 ratio

In [10]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([82]) target is tensor(46)
when input is tensor([82, 46]) target is tensor(63)
when input is tensor([82, 46, 63]) target is tensor(60)
when input is tensor([82, 46, 63, 60]) target is tensor(1)
when input is tensor([82, 46, 63, 60,  1]) target is tensor(42)
when input is tensor([82, 46, 63, 60,  1, 42]) target is tensor(73)
when input is tensor([82, 46, 63, 60,  1, 42, 73]) target is tensor(70)
when input is tensor([82, 46, 63, 60,  1, 42, 73, 70]) target is tensor(65)


block_size defines how many elements are considered in a sequence.<br><br>
x contains the first block_size elements.<br>
y contains the next block_size elements (shifted by one).<br><br>
For each step:

    The context grows in length from 1 to block_size elements.
    The target is always the next token in y.